In [5]:
import tensorflow as tf
import keras
import os

In [2]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

try:
    for i in gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

except Exception as e:
    print(f"{e}")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
BATCH_SIZE: int = 32
SEED: int = 123
LABEL_MODE: str = 'int'
IMAGE_SIZE: tuple = (224, 224)
INTERPOLATION: str = "bilinear"
COLOR: str = 'rgb'
LABELS: str = "inferred"

AUTOTUNE = tf.data.AUTOTUNE

In [7]:
path = "/mnt/d/Tojo Sir - Project/"
processed_path = os.path.join(path, "processed_data")

In [10]:
try:
    train_ds = tf.data.Dataset.load(os.path.join(processed_path, "train"))
    valid_ds = tf.data.Dataset.load(os.path.join(processed_path, "valid"))

except Exception as e:
    print(f"{e}")

tf.data snapshot element_spec file not found: /mnt/d/Tojo Sir - Project/processed_data/valid/dataset_spec.pb.


In [9]:
train = train_ds.prefetch(buffer_size = AUTOTUNE)
valid = valid_ds.prefetch(buffer_size = AUTOTUNE)

NameError: name 'valid_ds' is not defined

In [12]:
data_augmentation = keras.models.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomZoom(0.1, 0.1),
    keras.layers.RandomTranslation(0.1, 0.1),
    keras.layers.RandomRotation(0.2)
], name = "data_augmentation")

In [13]:
model = keras.models.Sequential([
    keras.layers.Input((IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),

    data_augmentation,

    keras.layers.Conv2D(16, (3,3), padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(activation=keras.activations.leaky_relu),
    keras.layers.MaxPool2D((2,2)),

    keras.layers.Conv2D(32, (3,3), padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(activation=keras.activations.leaky_relu),
    keras.layers.MaxPool2D((2,2)),

    keras.layers.Conv2D(64, (3,3), padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(activation=keras.activations.leaky_relu),
    keras.layers.MaxPool2D((2,2)),

    keras.layers.Flatten(),

    keras.layers.Dense(1024, activation = keras.activations.leaky_relu),
    keras.layers.Dense(256, activation = keras.activations.leaky_relu),
    keras.layers.Dense(64, activation = keras.activations.leaky_relu),
    keras.layers.Dense(4, activation = keras.activations.softmax)
])

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate= 0.1),
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    verbose = 1,
    restore_best_weights = True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.1,
    patience = 3,
    verbose = 1,
    min_lr = 0.00000001
)

In [16]:
model.fit(train,  validation_data = valid, epochs = 256, callbacks=[early_stopping])

NameError: name 'valid' is not defined